<a href="https://colab.research.google.com/github/csabi0312/crypto-svm/blob/gru/crypto14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install ujson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install oandapyV20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install v20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install --upgrade git+https://github.com/yhilpisch/tpqoa.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/yhilpisch/tpqoa.git to /tmp/pip-req-build-wb6yab83
  Running command git clone --filter=blob:none --quiet https://github.com/yhilpisch/tpqoa.git /tmp/pip-req-build-wb6yab83
  Resolved https://github.com/yhilpisch/tpqoa.git to commit f47a1ffddd2b2684c7e1569df6210b98dbe4e701
  Preparing metadata (setup.py) ... done


In [8]:
import pickle
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from numpy import trapz
import pickle
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from numpy import trapz
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import keras.losses
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras_tuner
import tpqoa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import load_model
import oandapyV20.endpoints.instruments as instruments
import datetime
import oandapyV20.endpoints.instruments as instruments
import datetime
from sklearn.metrics import mean_squared_error as mse

In [9]:
pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from rich.progress import track

# DATA

In [11]:
raw = pd.read_csv("/content/drive/MyDrive/Crypto/new3/full_spread_data.csv", parse_dates = ["time"], index_col = "time").dropna()
raw["returns"] =np.log(raw.Close / raw.Close.shift(1))
raw.drop(["Open","Spread"],axis=1,inplace=True)
cols = []
for lag in range(1, 40):
    col_h = "h_lag{}".format(lag)
    col_l = "l_lag{}".format(lag)
    col_c = "c_lag{}".format(lag)
    col_v = "v_lag{}".format(lag)
    raw[col_h] = raw.High.shift(lag)
    cols.append(col_h)
    raw[col_l] = raw.Low.shift(lag)
    cols.append(col_l)
    raw[col_c] = raw.Close.shift(lag)
    cols.append(col_c)
    raw[col_v] = raw.Volume.shift(lag)
    cols.append(col_v)
    raw.dropna(inplace = True)
raw.drop(["High","Low","Close","Volume"],axis=1,inplace=True)
raw

,returns,h_lag1,l_lag1,c_lag1,v_lag1,h_lag2,l_lag2,c_lag2,v_lag2,h_lag3,...,c_lag37,v_lag37,h_lag38,l_lag38,c_lag38,v_lag38,h_lag39,l_lag39,c_lag39,v_lag39
time,,,,,,,,,,,,,,,,,,,,,
2018-02-22 22:00:00+00:00,0.050812,10400.0,9601.5,9924.3,86014.0,10934.4,9738.6,9922.6,85916.0,11751.2,...,11590.0,85728.0,11805.4,9127.5,11371.0,86148.0,13905.9,10331.5,10713.0,85728.0
2018-02-23 22:00:00+00:00,-0.084014,10520.4,9924.3,10441.6,25177.0,10400.0,9601.5,9924.3,86014.0,10934.4,...,11351.7,85961.0,12108.9,10675.7,11590.0,85728.0,11805.4,9127.5,11371.0,86148.0
2018-02-24 22:00:00+00:00,0.077114,9868.7,9291.1,9600.2,60419.0,10520.4,9924.3,10441.6,25177.0,10400.0,...,12904.4,85853.0,12014.3,11052.6,11351.7,85961.0,12108.9,10675.7,11590.0,85728.0
2018-02-25 22:00:00+00:00,-0.007434,10434.4,9396.2,10369.8,85554.0,9868.7,9291.1,9600.2,60419.0,10520.4,...,11363.5,75479.0,13004.7,11243.7,12904.4,85853.0,12014.3,11052.6,11351.7,85961.0
2018-02-26 22:00:00+00:00,0.056381,10399.2,10114.4,10293.0,28880.0,10434.4,9396.2,10369.8,85554.0,9868.7,...,10358.4,85958.0,12961.8,11117.0,11363.5,75479.0,13004.7,11243.7,12904.4,85853.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-17 22:00:00+00:00,0.007969,21647.9,20407.7,20773.2,517432.0,21588.4,20860.6,21314.7,424610.0,21445.4,...,16194.6,291295.0,16617.2,16343.8,16505.0,205653.0,16784.7,16453.2,16541.2,244983.0
2023-01-18 22:00:00+00:00,0.064330,21195.0,20627.2,20939.4,399542.0,21647.9,20407.7,20773.2,517432.0,21588.4,...,16463.8,265767.0,16586.4,16003.0,16194.6,291295.0,16617.2,16343.8,16505.0,205653.0
2023-01-19 22:00:00+00:00,0.029523,22439.6,20865.0,22330.7,455144.0,21195.0,20627.2,20939.4,399542.0,21647.9,...,17103.6,252757.0,16528.6,16098.6,16463.8,265767.0,16586.4,16003.0,16194.6,291295.0


In [12]:
felso=np.quantile(raw["returns"],0.66)
also=np.quantile(raw["returns"],0.33)
print(felso,also)
returns2=(raw["returns"]>felso).astype(int)+2*(raw["returns"]<also).astype(int)
raw["returns2"]=returns2
to=int(len(raw)*0.7)
X_train=raw.drop(["returns","returns2"],axis=1).iloc[:to].to_numpy()
y_train=raw["returns2"].iloc[:to].to_numpy().astype(int)
X_test=raw.drop(["returns","returns2"],axis=1).iloc[to:].to_numpy()
y_test=raw["returns2"].iloc[to:].to_numpy().astype(int)

scaler1=MinMaxScaler()
X_train=scaler1.fit_transform(X_train)
X_test=scaler1.transform(X_test)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

0.010748025795442153 -0.009443080891668676
(923, 156) (923,) (396, 156) (396,)


# GRU

In [13]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [14]:
import tensorflow as tf
from keras.losses import binary_crossentropy, categorical_crossentropy
import keras.backend as K
import numpy as np
from prettytable import PrettyTable
from prettytable import ALL
from sklearn.metrics import f1_score

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [15]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(hp.Int('units', min_value=80, max_value=200, step=1),input_shape=(156,1)))
  model.add(Dropout(0.2))
  model.add(Dense(hp.Int('units1', min_value=15, max_value=120, step=1)))
  model.add(Dropout(0.2))
  model.add(GRU(units=hp.Int('units2', min_value=15, max_value=120, step=1), return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(LSTM(units=hp.Int('units3', min_value=15, max_value=120, step=1), return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(GRU(units=hp.Int('units2', min_value=15, max_value=120, step=1), return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(LSTM(units=hp.Int('units3', min_value=15, max_value=120, step=1), return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(GRU(units=hp.Int('units2', min_value=15, max_value=120, step=1), return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(LSTM(units=hp.Int('units3', min_value=15, max_value=120, step=1), return_sequences=False, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(Dense(hp.Int('units4', min_value=15, max_value=120, step=1)))
  model.add(Dropout(0.2))
  model.add(Dense(hp.Int('units5', min_value=80, max_value=200, step=1)))
  model.add(Dropout(0.2))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss=f1_loss, optimizer='adam', metrics=[f1])
  return model

In [16]:
tuner = keras_tuner.RandomSearch(
      build_model,
      objective='val_loss',
      max_trials=500,overwrite = True)
tuner.search(X_train,y_train , epochs=4, validation_split=0.2)
best_model = tuner.get_best_models()[0]
best_model.summary()

Trial 500 Complete [00h 00m 16s]
val_loss: 0.6830861568450928

Best val_loss So Far: 0.672756016254425
Total elapsed time: 02h 18m 07s
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 156, 155)          310       
                                                                 
 dropout (Dropout)           (None, 156, 155)          0         
                                                                 
 dense_1 (Dense)             (None, 156, 35)           5460      
                                                                 
 dropout_1 (Dropout)         (None, 156, 35)           0         
                                                                 
 gru (GRU)                   (None, 156, 43)           10320     
                                                                 
 dropout_2 (Dropout)         (None, 156, 43)         

In [40]:
def build_best():
  #hp_units = [hp.Int('units', min_value=160, max_value=320, step=1),hp.Int('units', min_value=16, max_value=120, step=1)]
  model = Sequential()
  model.add(Dense(156,input_shape=(156,1)))
  model.add(GRU(units=40, return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(GRU(units=40, return_sequences=True, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(GRU(units=40, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(Dense(156))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss=f1_loss, optimizer='adam', metrics=[f1])
  return model

In [84]:
#best_model=build_best()
early_stopping=EarlyStopping(patience=1000, verbose=0)
checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=1)
best_model.fit(X_train,y_train, batch_size=100,epochs=10000, verbose=0, \
                validation_split=0.1,callbacks=[checkpointer, early_stopping], shuffle=True)


Epoch 1: val_loss improved from inf to 0.66662, saving model to weights1.hdf5

Epoch 2: val_loss improved from 0.66662 to 0.63709, saving model to weights1.hdf5

Epoch 3: val_loss did not improve from 0.63709

Epoch 4: val_loss did not improve from 0.63709

Epoch 5: val_loss did not improve from 0.63709

Epoch 6: val_loss did not improve from 0.63709

Epoch 7: val_loss did not improve from 0.63709

Epoch 8: val_loss did not improve from 0.63709

Epoch 9: val_loss did not improve from 0.63709

Epoch 10: val_loss did not improve from 0.63709

Epoch 11: val_loss did not improve from 0.63709

Epoch 12: val_loss did not improve from 0.63709

Epoch 13: val_loss did not improve from 0.63709

Epoch 14: val_loss did not improve from 0.63709

Epoch 15: val_loss did not improve from 0.63709

Epoch 16: val_loss did not improve from 0.63709

Epoch 17: val_loss did not improve from 0.63709

Epoch 18: val_loss did not improve from 0.63709

Epoch 19: val_loss did not improve from 0.63709

Epoch 20: v

In [85]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
pred=best_model.predict(X_test)
y_pred = np.argmax(pred,1)
y_true = np.argmax(y_test,1)

print("Test accuracy: %g" %(accuracy_score(y_true, y_pred)))
print("Precision", precision_score(y_true, y_pred, average="macro"))
print("Recall", recall_score(y_true, y_pred, average="macro"))
print("f1_score", f1_score(y_true, y_pred, average="macro"))
print("\n Confusion matrix ")
conf=confusion_matrix(y_true, y_pred)
print(conf)

13/13 [==============================] - 2s 24ms/step
Test accuracy: 0.361111
Precision 0.24724036069651742
Recall 0.3648557213930348
f1_score 0.28695783270734604

 Confusion matrix 
[[ 51  74   0]
 [ 42  92   0]
 [ 35 102   0]]


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
pred=best_model.predict(X_train)
y_pred = np.argmax(pred,1)
y_true = np.argmax(y_train,1)

print("Test accuracy: %g" %(accuracy_score(y_true, y_pred)))
print("Precision", precision_score(y_true, y_pred, average="macro"))
print("Recall", recall_score(y_true, y_pred, average="macro"))
print("f1_score", f1_score(y_true, y_pred, average="macro"))
print("\n Confusion matrix ")
conf=confusion_matrix(y_true, y_pred)
print(conf)

29/29 [==============================] - 1s 18ms/step
Test accuracy: 0.373781
Precision 0.46377276294071484
Recall 0.3705486081758002
f1_score 0.278770336821244

 Confusion matrix 
[[278  30   2]
 [254  55   6]
 [244  42  12]]
